In [1]:

import pandas as pd

from src.io import load_results

pd.options.plotting.backend = "plotly"
from src.scoring import get_score_df, get_eligible_points_df, get_summary_df

In [2]:
result_df = load_results()
print(result_df)
print('cell type:', type(result_df.iloc[0, 0]))

score_df = get_score_df(result_df)
eligible_points_df = get_eligible_points_df(result_df)
summary_df = get_summary_df(result_df, score_df, eligible_points_df)
print(summary_df.round(2))

                x0_plus_x1_distribution_non_uniform_stat_dep                                                                                                                                                             \
                                                                                                                                                                                                                          
baseline_random  {'score': {'is_feature_importance_symmetric': ...                                                                                                                                                        
saabas           {'score': {'is_feature_importance_symmetric': ...                                                                                                                                                        
tree_shap        {'score': {'is_feature_importance_symmetric': ...                                                          

In [3]:
min_x_axis_value = .01
summary_df.loc[summary_df.time < min_x_axis_value, 'time'] = min_x_axis_value
summary_df.percentage = summary_df.percentage * 100
summary_df['explainer_name'] = summary_df.index

In [9]:
fig = summary_df.plot.scatter(x='time',
                              y='percentage',
                              # s='explainer_name', # https://www.geeksforgeeks.org/how-to-annotate-matplotlib-scatter-plots/
                              c='eligible_points',
                              text='explainer_name',
                              log_x=True,
                              labels={
                                  "time": "Time (seconds)",
                                  "percentage": "Percentage %",
                                  'eligible_points': 'maximum score',
                                  'explainer_name': 'Explainer'
                              },
                              title="Global performance of xAI methods",
                              )
# print(fig)
# must be an interactive plot
mask = paretoset(summary_df[['percentage', 'time']], sense=["max", "min"])
pareto_df = summary_df[mask].sort_values('time')

trace1 = pareto_df.plot.line(x='time',  #logx=True,
                             y='percentage',
                             # fig=fig,
                             )
fig.add_trace(trace1)
# plt.legend(['Pareto'], loc='lower right')
# plt.ylim([0, 1])
fig.show()

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [Figure({
    'data': [{'hovertemplate': 'time=%{x}<br>percentage=%{y}<extra></extra>',
              'legendgroup': '',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([1.00000000e-02, 2.23028660e-02, 8.53204727e-01, 1.05801201e+01,
                          2.12181497e+01]),
              'xaxis': 'x',
              'y': array([0.43669404, 0.60564568, 0.77218922, 0.83337746, 0.91330781]),
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'time'}},
               'yax

In [19]:
import plotly.express as px

import plotly.graph_objects as go

fig = go.Figure()

fig = px.scatter(summary_df,
                 x='time',
                 y='percentage',
                 # s='explainer_name', # https://www.geeksforgeeks.org/how-to-annotate-matplotlib-scatter-plots/
                 # c='eligible_points',
                 text='explainer_name',
                 log_x=True,
                 labels={
                     "time": "Time (seconds)",
                     "percentage": "Percentage %",
                     'eligible_points': 'maximum score',
                     'explainer_name': 'Explainer'
                 },
                 title="Global performance of xAI methods",
                 )
# fig.add_trace(px.line(pareto_df,
#                       x='time',  #logx=True,
#                       y='percentage', ))
fig.add_trace(go.Line(x=pareto_df.time,  #logx=True,
                      y=pareto_df.percentage,
                      ))
fig.show();


C:\Users\kbelaid\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


